# Prepare CMYP

In [1]:
import requests
import json

In [2]:
CMYP_USER = "brocmyp"
CMYP_PSW = "TeamPlanBYang$1"

kubectl -n eyicwux-test port-forward eric-sec-access-mgmt-0 8443:8443

only localhost can pass the https security check. 

In [3]:
def prepare_cmyp():
    """
    Prepare the cmyp user and connect to the cmyp server
    """
    # Create the cmyp user in LDAP via IAM
    token_query_url = ('https://localhost:8443/auth/realms/master/protocol/openid-connect/token')
    token_request_data = {'grant_type': 'password', 'username': 'adminplanb',
                          'password': 'kcpwplanb', 'client_id': 'admin-cli'}
    token_request_headers = {'content-type':
                             'application/x-www-form-urlencoded'}
    # Query the endpoint for the token
    token_request_response = requests.post(
        token_query_url, data=token_request_data, headers=token_request_headers,
        verify=False)
    token = json.loads(token_request_response.text)['access_token']
    user_creation_query_url = ('https://localhost:8443/'
                               'auth/admin/realms/local-ldap3/users')
    user_creation_request_data = {
        "username": CMYP_USER,
        "enabled": "true",
        "credentials": [
            {
                "type": "password",
                "value": CMYP_PSW
            }
        ],
        "attributes": {
            "uidNumber": ["710"],
            "roles": ["system-admin"],
            "pwdReset": ["FALSE"]
        }
    }
    user_creation_headers = {'content-type': 'application/json',
                             'Authorization': ''}
    user_creation_headers['Authorization'] = 'bearer ' + token
    # Send the create user request
    requests.post(
        user_creation_query_url, json=user_creation_request_data,
        headers=user_creation_headers, verify=False)

In [75]:
prepare_cmyp()

/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


# Set up the connection

kubectl -n eyicwux-test expose svc eric-cm-yang-provider --name=cmyp-svc --port=2222 --target-port=2222 --type=LoadBalancer

In [5]:
import paramiko
import os
import sys
import select
import time 

In [6]:
class SSHShell:
    def __init__(self, host, user, psw, port):
        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.ssh.connect(host, username=user, password=psw, port=port, timeout=2)
        self.refresh()

    def __del__(self):
        self.ssh.close()
    
    def refresh(self):
        self.channel = self.ssh.invoke_shell()
        self.stdin = self.channel.makefile('wb')
        self.stdout = self.channel.makefile('r')    

    def execute(self, cmd):
        cmd = cmd.strip('\n')
        self.stdin.write(cmd + '\n')
        self.stdin.flush()
        time.sleep(0.5)
        while self.stdout.channel.recv_ready():
            rl, wl, xl = select.select([ self.stdout.channel ], [ ], [ ])
            if len(rl) > 0:
                tmp = self.stdout.channel.recv(1024)
                output = tmp.decode()
                print(output)

In [76]:
CLI = SSHShell("10.120.217.249", "brocmyp", "TeamPlanBYang$1", 2222)

ssh -p 2222 -t brocmyp@10.120.217.249     
planbyang

In [77]:
CLI.execute('screen-length 100')

Welcome
Last login time: Fri Sep 16 17:23:53 2022
brocmyp connected from 10.120.217.52 using ssh on @
brocmyp@#screen-length 100
brocmyp@#


In [78]:
CLI.execute('autowizard false')

autowizard false
brocmyp@#


## BRM

In [53]:
CLI.execute('show running-config brm')

show running-config brm
% No entries found.
brocmyp@#


In [11]:
BRM = 'configuration-data'

In [8]:
VBRM = "DEFAULT-eric-data-document-database-pg"

## Backup

### List backups

In [127]:
CLI.execute(f'show brm backup-manager {BRM} backup')

show brm backup-manager configuration-data backup
% No entries found.
brocmyp@#


### Detail a backup

In [266]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1}')

show brm backup-manager DEFAULT backup cmypBackup01
backup cmypBackup01
 backup-name   cmypBackup01
 creation-time 2022-02-18T16:18:39.459365+00:00
 status        backup-complete
 creation-type manual
 progress-report 13398
  action-name                EXPORT
  additional-info            [ "Exported Backup: cmypBackup01-2022-02-18T16:18:39.459365Z.tar.gz" ]
  progress-percentage        100
  result                     success
  state                      finished
  time-action-started        2022-02-18T16:33:57.2157+00:00
  time-action-completed      2022-02-18T16:33:58.835425+00:00
  time-of-last-status-update 2022-02-18T16:33:58.835425+00:00
 sw-version "APR 201 34" 7.1.0-19
  product-name    "Document Database PG"
  production-date 2022-02-18T16:31:18.122509+00:00
  description     "eric-data-document-database-pg Agent for backup and restore."
  type            Database
 sw-version "APR XXX XX/X" R1A
  product-name    "Test Application"
  production-date 2019-05-17T14:12:44+00:00
  

### Get an attribute from a Backup

In [212]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1} sw-version')

show brm backup-manager DEFAULT backup cmypBackup01 sw-version

sw-version "APR 201 34" 7.1.0-19
 product-name    "Document Database PG"
 production-date 2022-02-18T16:18:39.579782+00:00
 description     "eric-data-document-database-pg Agent for backup and restore."
 type            Database
sw-version "APR XXX XX/X" R1A
 product-name    "Test Application"
 production-date 2019-05-17T14:12:44+00:00
 description     "Test Application Drop 15"
 type            Predelivery
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### create a backup

In [81]:
backupName1 = 'cmypBackup12'

In [78]:
backupName2 = 'cmypBackup04'

In [82]:
CLI.execute(f'brm backup-manager {BRM} create-backup name {backupName1}')

brm backup-manager configuration-data create-backup name cmypBackup12
return-value 30582
brocmyp@#


### Restore a backup

In [72]:
BRM = 'configuration-data'

In [85]:
CLI.execute(f'brm backup-manager {BRM} backup {backupName1} restore')

brm backup-manager configuration-data backup cmypBackup12 restore
return-value 30901
brocmyp@#Restore is about to start, sessions may be terminated soon!



In [18]:
CLI.execute(f'brm backup-manager DEFAULT-agent-three backup {backupName1} restore')

brm backup-manager DEFAULT-agent-three backup cmypBackup01 restore
return-value 45387
brocmyp@eric-cm-yang-provider-59df9f448d-gtthw#


In [156]:
CLI.execute(f'brm backup-manager {VBRM} backup {backupName1} restore')

Welcome
Last login time: First Login

brocmyp connected from 10.120.217.52 using ssh on eric-cm-yang-provider-5bbb75c878-jwdth
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#
brm backup-manager DEFAULT-eric-data-document-database-pg backup cmypBackup01 restore


return-value 14172
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#
Restore is about to start, sessions may be terminated soon!



### Export a backup

In [249]:
CLI.refresh()

In [218]:
uri = 'sftp://brsftp@bro-test-sftp:9022/bro_test'
pwd = "planbsftp"

In [220]:
CLI.execute(f'brm backup-manager {BRM} backup {backupName1} export uri {uri} password {pwd}')

brm backup-manager DEFAULT backup cmypBackup01 export uri sftp://brsftp@bro-test-sftp:9022/bro_test password planbsftp

return-value 13398
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### Delete a backup

In [144]:
CLI.execute(f'brm backup-manager {VBRM} delete-backup name {backupName2}')

brm backup-manager DEFAULT-eric-data-document-database-pg delete-backup name cmypBackup02

return-value 3819
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#


### Import a backup

In [150]:
CLI.execute(f"brm backup-manager {BRM} import-backup uri {uri}/{BRM}/{backupName1} password {pwd}")

brm backup-manager DEFAULT import-backup uri sftp://brsftp@bro-test-sftp:9022/bro_test/DEFAULT/cmypBackup01 password planbsftp


return-value 40760
brocmyp@eric-cm-yang-provider-5bbb75c878-jwdth#


## Housekeeping

### Get BRM's housekeeping

In [26]:
CLI.execute(f'show running-config brm backup-manager {BRM} housekeeping')

show running-config brm backup-manager configuration-data housekeeping
brm backup-manager configuration-data
 housekeeping max-stored-manual-backups 53
 housekeeping auto-delete  disabled
!
brocmyp@eric-cm-yang-provider-5f5c4fcbf8-cqt2c#


### Update housekeeping

In [184]:
CLI.execute(f'config')
CLI.execute(f'brm backup-manager {BRM} housekeeping max-stored-manual-backups 164')
CLI.execute(f'commit')
CLI.execute(f'exit')

config
Entering configuration mode private
brocmyp@(config)#
brm backup-manager configuration-data housekeeping max-stored-manual-backups 164
brocmyp@(config)#
commit
Commit complete.
brocmyp@(config)#
exit
brocmyp@#


In [16]:
CLI.execute(f'brm backup-manager {BRM} housekeeping auto-delete disabled')

brm backup-manager configuration-data housekeeping auto-delete disabled
brocmyp@eric-cm-yang-provider-5f5c4fcbf8-cqt2c(config)#


commit
Commit complete.
brocmyp@eric-cm-yang-provider-5f5c4fcbf8-cqt2c(config)#


## Scheduler

### show runing config

In [119]:
CLI.execute(f'show running-config brm backup-manager {BRM} scheduler')

show running-config brm backup-manager configuration-data scheduler
brm backup-manager configuration-data
 scheduler scheduled-backup-name SCHEDULED_BACKUP
 scheduler auto-export disabled
 scheduler admin-state unlocked
 scheduler periodic-event 2
 !
!
brocmyp@#


### show a periodic event

In [5]:
eventId = 1

In [10]:
CLI.execute(f'show running-config brm backup-manager {BRM} scheduler periodic-event {eventId}')

show running-config brm backup-manager configuration-data scheduler periodic-event 1
--------------------------------------------------------------------------------------------^
syntax error: element does not exist
brocmyp@#


### Update runing config

In [246]:
CLI.execute(f"""config 
            brm backup-manager DEFAULT scheduler admin-state locked
    brm backup-manager DEFAULT scheduler scheduled-backup-name BACKUP
    brm backup-manager DEFAULT scheduler auto-export enabled
    brm backup-manager DEFAULT scheduler auto-export-uri {uri}
    brm backup-manager DEFAULT scheduler auto-export-password {pwd}
    commit
    exit
    """)

config 
-------------------------------------------------------^
syntax error: unknown command
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#            brm backup-manager DEFAULT scheduler admin-state locked



In [19]:
CLI.execute(f"config") 

config
Entering configuration mode private
brocmyp@(config)#


In [194]:
CLI.execute(f"brm backup-manager DEFAULT scheduler auto-export-uri {uri}") 

brm backup-manager DEFAULT scheduler auto-export-uri sftp://brsftp@bro-test-sftp:9022/bro_test

brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


In [195]:
CLI.execute(f"brm backup-manager DEFAULT scheduler auto-export-password {pwd}") 

brm backup-manager DEFAULT scheduler auto-export-password planbsftp

brocmyp@eric-cm-yang-provider-5bbb75c878-629tm(config)#


In [43]:
CLI.execute(f'commit')

commit yes
The value must be one of:
  cancel  no  yes
Uncommitted changes found, commit them? [yes/no/CANCEL] 


### show BRM's scheduler

In [133]:
CLI.execute(f'show brm backup-manager {BRM} scheduler')

show brm backup-manager configuration-data scheduler
scheduler
 next-scheduled-time 2022-09-11T23:30:00.785717+00:00
brocmyp@#


### Show a periodic-event

In [166]:
CLI.execute(f'show running-config brm backup-manager {BRM} scheduler periodic-event {eventId}')

show running-config brm backup-manager configuration-data scheduler periodic-event 1
--------------------------------------------------------------------------------------------^
syntax error: element does not exist
brocmyp@#


### Show Scheduler Running Configuration

In [165]:
CLI.execute(f'show running-config brm backup-manager {BRM} scheduler')

show running-config brm backup-manager configuration-data scheduler
brm backup-manager configuration-data
 scheduler scheduled-backup-name SCHEDULED_BACKUP
 scheduler auto-export disabled
 scheduler admin-state unlocked
 scheduler periodic-event 1
 !
!
brocmyp@#


### create a scheduler

In [151]:
eventId = 1

In [152]:
CLI.execute(f"""config 
brm backup-manager {BRM} scheduler periodic-event {eventId}\
            weeks 1 days 1 hours 0 minutes 5 start-time 2022-06-16T00:35:00+02:00 \
            stop-time 2023-06-12T00:35:00+02:00
commit
exit
            """)

config 
Entering configuration mode private
brocmyp@(config)#brm backup-manager configuration-data scheduler periodic-event 1            weeks 1 days 1 hours 0 minutes 5 start-time 2022-06-16T00:35:00+02:00             stop-time 2023-06-12T00:35:00+02:00
brocmyp@(config)#commit
Commit complete.
brocmyp@(config)#exit
brocmyp@#            
brocmyp@#


In [105]:
CLI.execute(f"""config 
brm backup-manager {BRM} scheduler periodic-event {eventId}\
            weeks 1 days 1 hours 0 start-time 2022-06-16T00:35:00+02:00 \
            stop-time 2023-06-12T00:35:00+02:00
commit
exit
            """)

config 
Entering configuration mode private
brocmyp@(config)#brm backup-manager configuration-data scheduler periodic-event 5            weeks 1 days 1 hours 0 start-time 2022-06-16T00:35:00+02:00             stop-time 2023-06-12T00:35:00+02:00
Error:'brm backup-manager configuration-data scheduler periodic-event 5 start-time': Failed to convert value, database may be corrupt

brocmyp@(config)#commit
% No modifications to commit.
brocmyp@(config)#exit
brocmyp@#            
brocmyp@#


### update a sheduler event

In [163]:
CLI.execute(f"""config 
            brm backup-manager {BRM} scheduler periodic-event {eventId} \
            weeks 0 days 0 hours 24 minutes 7
    commit
    exit
    """)

config 
Entering configuration mode private
Current configuration users:
brocmyp ssh (cli from 10.120.217.52) on since 2022-09-13 13:53:27 private mode
brocmyp@(config)#            brm backup-manager configuration-data scheduler periodic-event 1             weeks 0 days 0 hours 24 minutes 7
brocmyp@(config)#    commit
Commit complete.
brocmyp@(config)#    exit
brocmyp@#    
brocmyp@#


In [164]:
CLI.execute(f"""config 
            brm backup-manager {BRM} scheduler periodic-event {eventId} \
            weeks 0 days 0 hours 24 minutes 7 
            brm backup-manager {BRM} scheduler periodic-event {eventId} \
            weeks 0 days 2 hours 11 minutes 8 
    commit
    exit
    """)

config 
Entering configuration mode private
Current configuration users:
brocmyp ssh (cli from 10.120.217.52) on since 2022-09-13 13:53:27 private mode
brocmyp@(config)#            brm backup-manager configuration-data scheduler periodic-event 1             weeks 0 days 0 hours 24 minutes 7 
brocmyp@(config)#            brm backup-manager configuration-data scheduler periodic-event 1             weeks 0 days 2 hours 11 minutes 8 
brocmyp@(config)#    commit
Commit complete.
brocmyp@(config)#    exit
brocmyp@#    
brocmyp@#


### show scheduler progress-report

In [214]:
CLI.execute(f'show brm backup-manager {BRM} scheduler progress-report 123')

show brm backup-manager DEFAULT scheduler progress-report 123

---------------------------------------------------------------------------------------------------------^
syntax error: unknown argument
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


## Action

### List Actions

In [213]:
CLI.execute(f'show brm backup-manager {BRM} action')

show brm backup-manager DEFAULT action


-------------------------------------------------------------------------------^
syntax error: unknown argument
brocmyp@eric-cm-yang-provider-5bbb75c878-629tm#


### Progress report

### show restore/export progress-report

In [39]:
CLI.execute(f'show brm backup-manager {BRM} backup {backupName1} progress-report 37004')

show brm backup-manager configuration-data backup cmypBackup05 progress-report 37004
progress-report 37004
 action-name                RESTORE
 progress-percentage        100
 result                     success
 result-info                "{Agent: eric-data-document-database-pg, Stage: PREPARATION, success: true, message: Preparation for restore is successful}, {Agent: more-large-agent, Stage: PREPARATION, success: true, message: Preparation for restore is successful}, {Agent: large-agent, Stage: PREPARATION, success: true, message: Preparation for restore is successful}, {Agent: large-agent, Stage: EXECUTION, success: true, message: The test service has completed restore of backup: cmypBackup05}, {Agent: eric-data-document-database-pg, Stage: EXECUTION, success: true, message: cmypBackup05 restore complete.}, {Agent: more-large-agent, Stage: EXECUTION, success: true, message: The test service has completed restore of backup: cmypBackup05}, {Agent: large-agent, Stage: POST_ACTIONS, suc

### show create import delete progress report

In [84]:
CLI.execute(f'show brm backup-manager {BRM} progress-report 30582')

show brm backup-manager configuration-data progress-report 30582
progress-report 30582
 action-name                CREATE_BACKUP
 progress-percentage        100
 result                     success
 result-info                "{Agent: eric-data-document-database-pg, Stage: PREPARATION, success: true, message: Preparation for backup is successful}, {Agent: large-agent, Stage: PREPARATION, success: true, message: Preparation for backup is successful}, {Agent: eric-data-document-database-pg, Stage: EXECUTION, success: true, message: The PG B&R Agent has completed a backup for cmypBackup12}, {Agent: large-agent, Stage: EXECUTION, success: true, message: The test service has completed a backup for cmypBackup12 and the data has been sent to the orchestrator}, {Agent: large-agent, Stage: POST_ACTIONS, success: true, message: Post backup actions completed}, {Agent: eric-data-document-database-pg, Stage: POST_ACTIONS, success: true, message: Post backup actions completed}"
 state                

# other command

In [216]:
CLI.refresh()

In [32]:
CLI.execute(f'commit')

commit
Commit complete.
brocmyp@(config)#


In [44]:
CLI.execute(f'exit')

exit
The value must be one of:
  cancel  no  yes
Uncommitted changes found, commit them? [yes/no/CANCEL] 


In [45]:
CLI.execute(f'yes')

yes
Commit complete.
brocmyp@#
